### Train with numpy

In [6]:
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    #  feed forward
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    loss = np.square(y_pred - y).sum()
    if t % 50 == 0:
        print(t, loss)
    
    # back propagation
    grad_y_pred = 2.0*(y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    

0 29187824.119592518
50 13029.543514578047
100 762.8457625555
150 88.88353310347081
200 12.41447494035343
250 1.8647846146848093
300 0.2930918497112106
350 0.04772821699169512
400 0.007996098166790502
450 0.0013705885609288447


### Train with pytorch

In [9]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # for GPU

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # feed forward
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
 
    # compute loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
        
    # back propagation
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    
    

99 1060.605712890625
199 18.94452476501465
299 0.46979472041130066
399 0.01286633312702179
499 0.000623706029728055


### pytorch with autograd

In [16]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # for GPU

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # feed forward
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
 
    # compute loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # back propagation
    
    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()
    
    # update weights
    # torch.no_grad()로 감싸는 이유 weight를 update하는 과정은 추적할 필요가 없기 때문
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

99 427.7125549316406
199 2.8303732872009277
299 0.03506159782409668
399 0.0008460583630949259
499 0.00010974017641274258


### pytorch with custom autograd function (forward, backward)

In [19]:
import torch

class MyReLU(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
    

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # for GPU

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    relu = MyReLU.apply
    
    y_pred = relu(x.mm(w1)).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

99 338.6495056152344
199 0.8415462970733643
299 0.0035408076364547014
399 0.00010826213110703975
499 2.180175215471536e-05


### pytorch with nn module

- nn module은 tensoflow의 keras처럼 상위 레벨의 뉴러 네트워크를 구성을 지원해주는 모듈

In [21]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(\
                           torch.nn.Linear(D_in, H),
                           torch.nn.ReLU(),
                           torch.nn.Linear(H, D_out))

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.8855626583099365
199 0.0570574551820755
299 0.0022355373948812485
399 0.00012033001985400915
499 7.823558917152695e-06


### pytorch with nn module + optim package

- optim package는 weight update를 optimizer에 따라 자동으로 해주는 패키지

In [24]:
import torch

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(\
                           torch.nn.Linear(D_in, H),
                           torch.nn.ReLU(), #여기에 custom을 추가하려면?
                           torch.nn.Linear(H, D_out))

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()

99 45.07582473754883
199 0.5264756679534912
299 0.0025126668624579906
399 6.261393082240829e-06
499 4.022768784750497e-09


### pytorch with custom nn module + optim package


In [25]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__() # TwoLayerNet이 부모의 method를 가져와 사용할 때, 그외의 상속관계를 고려하지 않아도 됨
        
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 3.287654399871826
199 0.051778510212898254
299 0.0015521328896284103
399 6.167944957269356e-05
499 2.822109536282369e-06


### pytorch with custom nn module + custom autograd function + optim package


In [6]:
import torch

class MyReLU(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__() # TwoLayerNet이 부모의 method를 가져와 사용할 때, 그외의 상속관계를 고려하지 않아도 됨
        
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        relu = MyReLU.apply
        h_relu = relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(10000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 1000 == 99:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 3.831587553024292
1099 4.932539932056557e-10
2099 5.791312568392826e-11
3099 2.4547780475003833e-11
4099 1.5122228122499415e-11
5099 9.972114280165645e-12
6099 6.94295489156449e-12
7099 5.215180717832446e-12
8099 4.505154929668187e-12
9099 3.897319099388508e-12


### pytorch with weight sharing


In [8]:
import torch
import random

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__() 
        # TwoLayerNet이 부모의 method를 가져와 사용할 때, 그외의 상속관계를 고려하지 않아도 됨
        
        self.input_layer = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_layer = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.input_layer(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_layer(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(10000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 1000 == 99:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 110.77033233642578
1099 0.7475787997245789
2099 0.3605017364025116
3099 0.014393921941518784
4099 0.007182911038398743
5099 0.00393475079908967
6099 0.10295569896697998
7099 0.023540791124105453
8099 0.017527412623167038
9099 0.052477169781923294
